## Abstract
Despite the fact that the agricultural sector is a national economic development priority in sub-Saharan Africa, crop pests and diseases have been the challenge affecting major food security crops like maize. 
Maize Leaf Blight, also known as Northern Corn Leaf Blight has become a menace in low land agro-ecologies, during the last decade. On the other hand, according to research, Maize Streak Disease which is caused by the Maize Streak Virus is regarded as the third most serious disease affecting maize in sub-Saharan Africa. 
The prominence of these diseases has greatly affected the yields of Africa’s most important food crop. 

## Classes
<div style="display: flex; justify-content: center;">
    <div style="text-align: center; margin-right: 20px;">
        <p>Healthy</p>
        <img src="sample/1621590060253.jpg" alt="Image 1" style="width: 200;">
    </div>
    <div style="text-align: center;">
        <p>Maize Streak Blight</p>
        <img src="sample/1621319276554.jpg" alt="Image 2" style="width: 200;">
    </div>
</div>


In [38]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
import keras
import numpy as np
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [39]:
# params
LEARNING_RATE = 0.01
CLASSES = 2
EPOCHS = 2
INCLUDE_TOP = False
BATCH_SIZE = 16
AUGMENTATION = True
WEIGHTS = "imagenet"
IMAGE_SIZE = [224, 224, 3]
DECAY = 1e-6
MOMENTUM = 0.9

In [40]:
# data path
DATA = "data/"

In [41]:
# base model
base_model = VGG16(input_shape=IMAGE_SIZE, weights=WEIGHTS, include_top=INCLUDE_TOP)


In [42]:
# base model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [43]:
# freeze hidden layers
for layer in base_model.layers:
    layer.trainable = False

In [44]:
# create custom output(dense) layer
output = Dense(CLASSES, activation="softmax") (Flatten() (base_model.output))

In [45]:
# custom model
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(
            optimizer=optimizers.SGD(learning_rate=LEARNING_RATE, decay=DECAY, momentum=MOMENTUM),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
new_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [46]:
# Augmentation
datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )
dataflow_kwargs = dict(
            target_size=IMAGE_SIZE[:-1],
            batch_size=BATCH_SIZE,
            interpolation="bilinear"
        )
train_datagenerator = image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=20,
                height_shift_range=20,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
validation_datagenerator = image.ImageDataGenerator(
               **datagenerator_kwargs
            )



In [47]:
# train test sets
train_set = train_datagenerator.flow_from_directory(
            directory=DATA,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
validation_set = validation_datagenerator.flow_from_directory(
            directory=DATA,
            subset='validation',
            shuffle=True,
            **dataflow_kwargs
        )

Found 960 images belonging to 2 classes.
Found 240 images belonging to 2 classes.
